# GUI For Power Load Prediction

Hint : You have to download all 'saved_models'  

In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
import pandas as pd
import joblib
import os

def create_dual_gui():
    
    def load_models(load_type):
        try:
            models_dir = "saved_models"
            suffix = f"{load_type}_load"
            
            median_model = joblib.load(os.path.join(models_dir, f'median_model_{suffix}.pkl'))
            quantile_lower = joblib.load(os.path.join(models_dir, f'quantile_lower_model_{suffix}.pkl'))
            quantile_upper = joblib.load(os.path.join(models_dir, f'quantile_upper_model_{suffix}.pkl'))
            model_metadata = joblib.load(os.path.join(models_dir, f'model_metadata_{suffix}.pkl'))
            
            return median_model, quantile_lower, quantile_upper, model_metadata
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load {load_type} load models: {str(e)}")
            return None, None, None, None
    
    current_values = {}
    
    suggested_values = {
        "Wall Area": ["294", "318.5", "343", "416.5", "245", "269.5", "367.5", "Other"],
        "Roof Area": ["110.25", "122.5", "147", "220.5", "Other"],
        "Overall Height": ["7", "3.5", "Other"],
        "Orientation": ["2", "3", "4", "5", "Other"],
        "Glazing Area": ["0", "0.1", "0.25", "0.4", "Other"],
        "Glazing Area Distribution": ["0", "1", "2", "3", "4", "5", "Other"]
    }
    
    def on_combobox_select(event, feature_name, combobox, entry_var, entry_widget):
        if combobox.get() == "Other":
            entry_var.set("")
            combobox.config(state='disabled')
            entry_widget.config(state='normal', background='white')
            entry_widget.focus()
        else:
            entry_var.set(combobox.get())
            combobox.config(state='readonly')
            entry_widget.config(state='disabled', background='white')
    
    def on_entry_focusout(event, feature_name, combobox, entry_var, entry_widget):
        if entry_var.get().strip() == "":
            entry_var.set("")
            combobox.config(state='readonly')
            entry_widget.config(state='disabled', background='white')
            combobox.set(suggested_values[feature_name][0])
    
    def predict_load():
        try:
            load_type = load_type_var.get().lower()
            
            median_model, quantile_lower, quantile_upper, model_metadata = load_models(load_type)
            if median_model is None:
                return
            
            feature_names = model_metadata['feature_names']
            
            input_values = {}
            for i, feature in enumerate(features):
                if feature in suggested_values:
                    value = entry_vars[feature].get()
                else:
                    value = entries[i].get()
                
                if not value:
                    messagebox.showerror("Error", "Please fill all fields")
                    return
                try:
                    input_values[feature_names[i]] = float(value)
                    current_values[feature] = value  # Store the value
                except ValueError:
                    messagebox.showerror("Error", "Please enter valid numbers")
                    return
            
            input_df = pd.DataFrame([input_values], columns=feature_names)
            
            median_pred = median_model.predict(input_df)[0]
            lower_pred = quantile_lower.predict(input_df)[0]
            upper_pred = quantile_upper.predict(input_df)[0]
            
            prediction_result.config(text=f"{median_pred:.4f}")
            lower_limit_result.config(text=f"{lower_pred:.4f}")
            upper_limit_result.config(text=f"{upper_pred:.4f}")
            load_type_label.config(text=f"{load_type.capitalize()} Load Prediction:")
            
        except Exception as e:
            messagebox.showerror("Error", f"Prediction failed: {str(e)}")
    
    def clear_fields():
        for entry in entries:
            entry.delete(0, tk.END)
            entry.insert(0, "0.0")
            entry.config(background='white')
        
        for feature in suggested_values:
            comboboxes[feature].set(suggested_values[feature][0])
            entry_vars[feature].set(suggested_values[feature][0])
            comboboxes[feature].config(state='readonly')
            entry_widgets[feature].config(state='disabled', background='white')
        
        prediction_result.config(text="")
        lower_limit_result.config(text="")
        upper_limit_result.config(text="")
        current_values.clear()
    
    def update_load_type():
        load_type = load_type_var.get().lower()
        load_type_label.config(text=f"{load_type.capitalize()} Load Prediction:")
        
        prediction_result.config(text="")
        lower_limit_result.config(text="")
        upper_limit_result.config(text="")
    
    root = tk.Tk()
    root.title("Building Load Prediction System")
    root.geometry("790x760")
    root.configure(bg='white')
    root.resizable(False, False)
    
    style = ttk.Style()
    style.theme_use('clam')
    
    style.configure('TFrame', background='white')
    style.configure('TLabel', background='white', font=('Segoe UI', 10))
    style.configure('Title.TLabel', background='white', font=('Segoe UI', 16, 'bold'))
    style.configure('Header.TLabel', background='white', font=('Segoe UI', 12, 'bold'))
    style.configure('TButton', font=('Segoe UI', 10))
    style.configure('Predict.TButton', background='#28a745', foreground='white')
    style.configure('Clear.TButton', background='#dc3545', foreground='white')
    style.configure('TCombobox', fieldbackground='white', background='white')
    style.configure('TEntry', fieldbackground='white')
    
    style.map('TCombobox', 
              fieldbackground=[('readonly', 'white'), ('disabled', 'white')],
              background=[('readonly', 'white'), ('disabled', 'white')])
    
    main_container = ttk.Frame(root, padding="20")
    main_container.pack(fill='both', expand=True)
    
    title_label = ttk.Label(main_container, text="Building Load Prediction System", 
                           style='Title.TLabel')
    title_label.pack(pady=(0, 20))
    
    type_frame = ttk.Frame(main_container)
    type_frame.pack(fill='x', pady=(0, 20))
    
    ttk.Label(type_frame, text="Select Load Type:", font=('Segoe UI', 11), 
             style='Header.TLabel').pack(side='left', padx=(0, 10))
    
    load_type_var = tk.StringVar(value="Heating")
    load_type_combo = ttk.Combobox(type_frame, textvariable=load_type_var, 
                                  values=["Heating", "Cooling"], state="readonly", width=12)
    load_type_combo.pack(side='left')
    load_type_combo.bind('<<ComboboxSelected>>', lambda e: update_load_type())
    
    input_card = ttk.Frame(main_container, relief='raised', borderwidth=1)
    input_card.pack(fill='x', pady=(0, 20))
    
    input_header = ttk.Frame(input_card)
    input_header.pack(fill='x', pady=(10, 5))
    ttk.Label(input_header, text="Input Parameters", style='Header.TLabel').pack(side='left', padx=10)
    
    input_fields = ttk.Frame(input_card)
    input_fields.pack(fill='x', padx=10, pady=10)
    
    features = [
        "Relative Compactness",
        "Surface Area",
        "Wall Area", 
        "Roof Area",
        "Overall Height",
        "Glazing Area",
        "Orientation",
        "Glazing Area Distribution"
    ]
    
    defaults = [0.75, 500, 294, 110.25, 7, 0, 2, 0]
    
    entries = []
    comboboxes = {}
    entry_vars = {}
    entry_widgets = {}
    
    for i, feature in enumerate(features):
        row = i % 4
        col = i // 4
        
        frame = ttk.Frame(input_fields)
        frame.grid(row=row, column=col, padx=10, pady=8, sticky='w')
        
        label = ttk.Label(frame, text=feature + ":", width=22, anchor='w')
        label.pack(side='left')
        
        if feature in suggested_values:
            input_container = ttk.Frame(frame)
            input_container.pack(side='right')
            
            entry_vars[feature] = tk.StringVar(value=str(defaults[i]))
            
            combobox = ttk.Combobox(input_container, textvariable=entry_vars[feature], 
                                   values=suggested_values[feature], state="readonly", width=12)
            combobox.pack(side='left')
            comboboxes[feature] = combobox
            
            entry = tk.Entry(input_container, textvariable=entry_vars[feature], width=12, font=('Segoe UI', 10), 
                           state='disabled', bg='white', relief='sunken')
            entry.pack(side='left')
            entry_widgets[feature] = entry
            
            combobox.bind('<<ComboboxSelected>>', 
                         lambda e, f=feature: on_combobox_select(e, f, comboboxes[f], entry_vars[f], entry_widgets[f]))
            
            entry.bind('<FocusOut>', 
                      lambda e, f=feature: on_entry_focusout(e, f, comboboxes[f], entry_vars[f], entry_widgets[f]))
            
            entries.append(tk.Entry())  
        else:
            entry = tk.Entry(frame, width=12, font=('Segoe UI', 10), bg='white')
            entry.insert(0, str(defaults[i]))
            entry.pack(side='right')
            entries.append(entry)
    
    button_frame = ttk.Frame(main_container)
    button_frame.pack(pady=(0, 20))
    
    predict_btn = ttk.Button(button_frame, text="Predict", command=predict_load,
                            style='Predict.TButton', width=12)
    predict_btn.pack(side='left', padx=5)
    
    clear_btn = ttk.Button(button_frame, text="Clear", command=clear_fields,
                          style='Clear.TButton', width=12)
    clear_btn.pack(side='left', padx=5)
    
    result_card = ttk.Frame(main_container, relief='raised', borderwidth=1)
    result_card.pack(fill='x', pady=(0, 20))
    
    result_header = ttk.Frame(result_card)
    result_header.pack(fill='x', pady=(10, 5))
    load_type_label = ttk.Label(result_header, text="Heating Load Prediction:", 
                               style='Header.TLabel')
    load_type_label.pack(side='left', padx=10)
    
    result_grid = ttk.Frame(result_card)
    result_grid.pack(fill='x', padx=20, pady=10)
    
    ttk.Label(result_grid, text="Prediction:", font=('Segoe UI', 10)).grid(
        row=0, column=0, sticky='w', pady=8)
    prediction_result = ttk.Label(result_grid, text="", background='white', 
                                 relief='sunken', width=15, font=('Segoe UI', 10),
                                 anchor='center')
    prediction_result.grid(row=0, column=1, padx=10, pady=8)
    
    ttk.Label(result_grid, text="Lower Limit:", font=('Segoe UI', 10)).grid(
        row=1, column=0, sticky='w', pady=8)
    lower_limit_result = ttk.Label(result_grid, text="", background='white', 
                                  relief='sunken', width=15, font=('Segoe UI', 10),
                                  anchor='center')
    lower_limit_result.grid(row=1, column=1, padx=10, pady=8)
    
    ttk.Label(result_grid, text="Upper Limit:", font=('Segoe UI', 10)).grid(
        row=2, column=0, sticky='w', pady=8)
    upper_limit_result = ttk.Label(result_grid, text="", background='white', 
                                  relief='sunken', width=15, font=('Segoe UI', 10),
                                  anchor='center')
    upper_limit_result.grid(row=2, column=1, padx=10, pady=8)
    
    info_frame = ttk.Frame(main_container, relief='raised', borderwidth=1)
    info_frame.pack(fill='x', pady=(0, 10))
    
    info_header = ttk.Frame(info_frame)
    info_header.pack(fill='x', pady=(10, 5))
    ttk.Label(info_header, text="Prediction Information:", style='Header.TLabel').pack(side='left', padx=10)
    
    info_content = ttk.Frame(info_frame)
    info_content.pack(fill='x', padx=15, pady=(0, 10))
    
    info_text = """• Prediction: Most likely value (50th percentile)
• Lower Limit: 5th percentile - lower bound estimate
• Upper Limit: 95th percentile - upper bound estimate
• Input values are preserved when switching load types
• For categorical parameters, select from suggested values or choose 'Other' to enter a custom value"""
    
    info_label = ttk.Label(info_content, text=info_text, justify='left', 
                          font=('Segoe UI', 9))
    info_label.pack(anchor='w', pady=5)
    

    root.mainloop()


create_dual_gui()